## SIMPLEX
### Optimització - Pràctica 1

In [15]:
import numpy as np

# Función para leer el archivo y extraer las matrices
def leer_archivo(nombre_archivo):
    c = []
    A = []
    b = []

    with open(nombre_archivo, 'r') as file:
        for line in file:
            if 'c=' in line:
                while True:
                    line = next(file)
                    c.append([int(x) for x in line.split()])
                    if line=='\n':
                        c.pop()                      
                        break 
            elif 'A=' in line:
                while True:
                    line = next(file)                    
                    A.append([int(x) for x in line.split()])
                    if line=='\n':  
                        A.pop()                    
                        break 

            elif 'b=' in line:
                while True:
                    line = next(file)
                    b.append([int(x) for x in line.split()])
                    if line=='\n': 
                        b.pop()                     
                        break 
    # Convertir las listas a arrays numpy
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    
    return c, A, b

# Nombre del archivo
nombre_archivo = 'tes.txt'

# Llamar a la función para leer el archivo
c, A, b = leer_archivo(nombre_archivo)

# Mostrar las matrices
print("Matriz c:")
print(c)
print("\nMatriz A:")
print(A)
print("\nVector b:")
print(b)

Matriz c:
[[-85 -65 -34  92 -83  93  22  76  48 -71  25 -50  79 -99   0   0   0   0
    0   0]]

Matriz A:
[[-13 -29  23 -27  57 -52 -70  81  -2  64 -54   9  68  10   0   0   0   0
    0   0]
 [ -6 -53 -37  36  85 -22  35  87  32  34  65 -64  65 -25   0   0   0   0
    0   0]
 [-13  93  31  16  45  23 -11  83  75 -65  67  10 -92  88   0   0   0   0
    0   0]
 [ 67  -8 -27 -22  47  44  -3 -23 -16  81 -24 -38 -67  24   0   0   0   0
    0   0]
 [ 80  57  77  72  52  58  53  84  77  92  57  75  68  88   1   0   0   0
    0   0]
 [-81  21 -14  73  11 -10  -7  45  95 -92  97 -79  89 -92   0  -1   0   0
    0   0]
 [ 96  -7  85 -63 -24 -95  42 -71 100  42  43 -23  83  60   0   0  -1   0
    0   0]
 [-27  92 -84  77  68  73  28  53  34 -71  59 -17 -36  -1   0   0   0   1
    0   0]
 [ -1 -28  96  74  66 -82 -24  91  63  65  64 -29  73  93   0   0   0   0
   -1   0]
 [-60 -34  21 -65  84   4  12  98 -38  -1 -99  25  21  82   0   0   0   0
    0  -1]]

Vector b:
[[ 65 232 350  35 991  55 267 2

Classe

Queda: preguntar por la lectura de los problemas, la actualización de la inversa y preguntar por la fase 1 cálculo de la SBF

generas el nuevo problema fase 1 encuentras su óptimo y la base resultante es la SBF de la siguiente parte.

In [16]:
import numpy as np
class SIMPLEX:
    def __init__(self,Coeficientes, Arestricciones, bsolucion):
        self.__c=Coeficientes
        self.__A=Arestricciones
        self.__b=bsolucion

    def matriz_B(self, fase1=False):
        if fase1:
            self.B = self.A_fase1[:, self.Base]
        else:
            self.B = self.__A[:, self.Base]

    def matriz_An(self, fase1=False):
        if fase1:
            self.An = self.A_fase1[:, self.NoBase]
        else:
            self.An = self.__A[:, self.NoBase]

    def Cb_Cn(self, fase1=False):
        if fase1:
            self.Cb = self.c_fase1[self.Base]
            self.Cn = self.c_fase1[self.NoBase]

        else:
            self.Cb = self.__c[self.Base]
            self.Cn = self.__c[self.NoBase]

    def Binversa(self, act=False):
        if act:
            print('nada todavia')
        else:
            self.Binv = np.linalg.pinv(self.B)

    def A_base_nobase_fase1(self):
        m, n = self.__A.shape
        self.A_fase1 = np.zeros((m, m + n))
        self.A_fase1[:, :n] = self.__A
        np.fill_diagonal(self.A_fase1[:, n:], 1)
        self.Base = list(range(n, n + m))
        self.NoBase = list(range(n))
    def construct_c_fase1(self):
        num_artificial_variables = self.__A.shape[0]
        num_original_variables = self.__c.shape[0]
        self.c_fase1 = np.concatenate((np.zeros(num_original_variables), np.ones(num_artificial_variables)))

    def menor_indice_r(self):
        i=0
        menor_indice=99999
        while i<len(self.r):
            if (self.NoBase[i]< menor_indice) and (self.r[i]<0):
                menor_indice=self.NoBase[i]
                self.q_r=i

            i+=1
        self.q = menor_indice
    def calc_theta(self):
        minimo=999
        for i in range(len(self.db)):
            self.theta=(-self.Xb[i] / self.db[i])
            if self.theta<minimo:
                minimo=self.theta
                self.sale=i 

    def fase1_SIMPLEX(self):
        self.A_base_nobase_fase1()
        self.construct_c_fase1()
        while True:
            self.matriz_B(fase1=True)
            self.matriz_An(fase1=True)
            self.Cb_Cn(fase1=True)
            self.Binversa()

            self.Xb = np.dot(self.Binv, self.__b)
            self.Xn = np.zeros(len(self.NoBase))

            self.z = np.dot(self.Cb, self.Xb) + np.dot(self.Cn, self.Xn)
            self.r = self.Cn - np.dot(np.dot(self.Cb, self.Binv), self.An)
            print('fase1',self.z)

            self.menor_indice_r()

            if np.all(self.r >= 0):
                if round(self.z, 4) > 0:
                    return 'No factible'
                elif round(self.z, 4) == 0:
                    return self.calcula_óptimo_()

            Aq = self.A_fase1[:, self.q]
            self.db = np.dot(self.Binv, Aq)
            if np.all(self.db >= 0):
                return 'No acotado'

            self.calc_theta()

            sale_ = self.Base[self.sale]
            self.Base[self.sale] = self.NoBase[self.q_r]
            self.NoBase[self.q_r] = sale_
    
    def calcula_óptimo_(self):
        """
        Se calcula todo aquí pero podría hacerse en un método a parte de inicialización
        """
        while True:
            self.matriz_B()
            self.matriz_An()
            self.Cb_Cn()
            self.Binversa()
            self.Xb = np.dot(self.Binv, self.__b)
            self.Xn = np.zeros(len(self.NoBase))

            self.z = np.dot(self.Cb, self.Xb) + np.dot(self.Cn, self.Xn)
            self.r = self.Cn - np.dot(np.dot(self.Cb, self.Binv), self.An)
            print(self.z)
            self.menor_indice_r()

            if np.all(self.r >= 0):
                return self.Xb, self.Xn, self.z

            Aq = self.__A[:, self.q]
            self.db = np.dot(self.Binv, Aq)
            if np.all(self.db >= 0):
                return 'No acotado'

            self.calc_theta()

            sale_ = self.Base[self.sale]
            self.Base[self.sale] = self.NoBase[self.q_r]
            self.NoBase[self.q_r] = sale_
            # Actualiza self.Binv con la nueva matriz B
            #self.Xb = self.Xb +  self.theta * self.db
            #self.B = np.array([self.__A[:, j] for j in self.Base]).T
            #self.An = np.array([self.__A[:, j] for j in self.NoBase]).T
            #self.Binv = np.linalg.pinv(self.B)



SImplex

In [ ]:
import numpy as np

class Simplex:
    def __init__(self, c, A, b):
        self.c = c
        self.A = A
        self.b = b
        self.m, self.n = A.shape
    def calcula(self):
        # Fase 1: Introducir variables artificiales
        c_fase1 = np.concatenate((np.zeros(self.n), np.ones(self.m)))
        A_fase1 = np.hstack((self.A, np.eye(self.m)))
        self.base = np.arange(self.n, self.n + self.m)
        self.no_base = np.arange(self.n)

        # Resolver problema auxiliar
        resolvemos_fase1 = self.SIMPLEX(A_fase1, b, c_fase1)
        print(resolvemos_fase1[0])
        if resolvemos_fase1[0] != 0:
            return "No hay solución factible"

        # Fase 2: Resolver problema original
        return self.SIMPLEX(self.A, self.b, self.c)

    def SIMPLEX(self, A, b, c):
        while True:
            print('la base',self.base)
            print('la no base',self.no_base)

            B = A[:, self.base]
            An = A[:, self.no_base]
            B_inv = np.linalg.inv(B)
            x_b = np.dot(B_inv, b)
            x_n=np.zeros(len(self.no_base))
            c_b = c[self.base]
            c_n = c[self.no_base]
           
            r = np.array(c_n - np.dot(np.dot(c_b, B_inv),An))
            z = np.dot(c_b, x_b)+np.dot(c_n, x_n)
            if np.all(r >= 0):
                print(np.all>=0)
                # Óptimo encontrado
                return [z, x_b,x_n]
            indice_variable_entra = np.where(r < 0)[0][0]
            print('entra la variable',self.no_base[indice_variable_entra])
            d_b = np.dot(-B_inv, A[:, self.no_base[indice_variable_entra]])
            if np.all(d_b <= 0):
                return "No acotado"
            print('resultado z',z)
            theta = np.divide(-x_b, d_b)
            indice_variable_sale = np.argmin(theta)

            variable_sale = self.base[indice_variable_sale]
            self.base[indice_variable_sale] = self.no_base[indice_variable_entra]
            self.no_base[indice_variable_entra] = variable_sale
            self.no_base.sort()

In [19]:
c= np.array([-85, -65, -34, 92, -83, 93, 22, 76, 48, -71, 25, -50, 79, -99, 0, 0, 0, 0, 0, 0])
b = np.array([65, 232, 350, 35, 991, 55, 267, 249, 520, 49])
A=np.array([[-13, -29, 23, -27, 57, -52, -70, 81, -2, 64, -54, 9, 68, 10, 0, 0, 0, 0, 0, 0],
   [-6, -53, -37, 36, 85, -22, 35, 87, 32, 34, 65, -64, 65, -25, 0, 0, 0, 0, 0, 0],
   [-13, 93, 31, 16, 45, 23, -11, 83, 75, -65, 67, 10, -92, 88, 0, 0, 0, 0, 0, 0],
   [67, -8, -27, -22, 47, 44, -3, -23, -16, 81, -24, -38, -67, 24, 0, 0, 0, 0, 0, 0],
   [80, 57, 77, 72, 52, 58, 53, 84, 77, 92, 57, 75, 68, 88, 1, 0, 0, 0, 0, 0],
   [-81, 21, -14, 73, 11, -10, -7, 45, 95, -92, 97, -79, 89, -92, 0, -1, 0, 0, 0, 0],
   [96, -7, 85, -63, -24, -95, 42, -71, 100, 42, 43, -23, 83, 60, 0, 0, -1, 0, 0, 0],
   [-27, 92, -84, 77, 68, 73, 28, 53, 34, -71, 59, -17, -36, -1, 0, 0, 0, 1, 0, 0],
   [-1, -28, 96, 74, 66, -82, -24, 91, 63, 65, 64, -29, 73, 93, 0, 0, 0, 0, -1, 0],
   [-60, -34, 21, -65, 84, 4, 12, 98, -38, -1, -99, 25, 21, 82, 0, 0, 0, 0, 0, -1]])

In [17]:
simplex = Simplex(c, A, b)
result = simplex.calcula()
print("Resultado:", result)

In [49]:

c = np.array([2, 3,0,0])
A = np.array([[3, 4,1,0], [2, 5,0,1]])
b = np.array([12, 10])

simplex = Simplex(c, A, b)
result = simplex.calcula()
print("Resultado:", result)


la base [4 5]
la no base [0 1 2 3]
entra la variable 0
N
Resultado: No hay solución factible
